Step 1 - Scraping

In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import tweepy
import time
import pandas as pd

In [2]:
#Initiate browser helper function
def init_browser():
    exec_path = {'executable_path': '/usr/local/bin/chromedriver'}
    return Browser('chrome', **exec_path, headless=True)

In [3]:
#List of URLs to explore and scrape
url1 = 'https://mars.nasa.gov/news/'
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url3 = 'https://twitter.com/marswxreport?lang=en'
url4 = 'http://space-facts.com/mars/'
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

NASA Mars News

In [4]:
# Mars News URL (url1)
browser = init_browser()
browser.visit(url1)

In [5]:
# Retrieve page with the requests module
html = requests.get(url1)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html.text, 'html.parser')

# Get title & description
news_title_1 = soup.find('div', 'content_title', 'a').text
news_p_1 = soup.find('div', 'rollover_description_inner').text

In [6]:
news_title = news_title_1.strip()
news_title

'Opportunity Hunkers Down During Dust Storm'

In [7]:
news_p = news_p_1.strip()
news_p

"It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home."

JPL Mars Space Images

In [8]:
# Mars Featured Image (url2)
browser = init_browser()
browser.visit(url2)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
expand = browser.find_by_css('a.fancybox-expand')
expand.click()
time.sleep(1)

jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

img_relative = jpl_soup.find('img', class_='fancybox-image')['src']
featured_image_url = f'https://www.jpl.nasa.gov{img_relative}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17440_ip.jpg


Mars Weather

In [9]:
# Mars Weather URL (url3)
browser = init_browser()
browser.visit(url3)

In [10]:
# Get weather using BeautifulSoup
html_weather = browser.html
soup = BeautifulSoup(html_weather, "html.parser")
for text in browser.find_by_css('.tweet-text'):
    if text.text.partition(' ')[0] == 'Sol':
        mars_weather = text.text
        break
print(mars_weather)

Sol 2108 (2018-07-12), Sunny, high -24C/-11F, low -65C/-84F, pressure at 8.06 hPa, daylight 05:19-17:27


Mars Facts

In [11]:
# Mars Facts URL (url4)
browser = init_browser()
browser.visit(url4)
time.sleep(1)
mars_facts_html = browser.html
mars_facts_soup = BeautifulSoup(mars_facts_html, 'html.parser')

fact_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts

,Facet,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
# Mars Hemispheres URL & use of Python dictionary to store data (url5)
hemi_dicts = []

for i in range(1,9,2):
    hemi_dict = {}
    
    browser.visit(url5)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemi_img_soup = BeautifulSoup(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path

    hemi_dicts.append(hemi_dict)

Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


Step 2 - MongoDB and Flask Application

Convert Jupyter notebook into a Python script called scrape_mars.py